In [1]:
!nvidia-smi

Wed Apr  3 11:56:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:22:00.0 Off |                    0 |
| N/A   31C    P0              54W / 300W |      7MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os, torch

print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

True
Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [8]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel

controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble",
                                                  torch_dtype=torch.float16,
                                                  use_safetensors=True)
pipe = StableDiffusionControlNetPipeline.from_pretrained("dreamlike-art/dreamlike-photoreal-2.0",
                                                         controlnet = controlnet,
                                                         torch_dtype=torch.float16,
                                                        use_safetensors=True).to('cuda')
pipe.save_pretrained('pipe-controlnet-dreamlike')

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
from controlnet import ControlNetAslan
module = ControlNetAslan('pipe-controlnet-dreamlike')

/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [41]:
# without lora 768 x 512
module.run( 'front1.png', # 샘플 이미지
            'bmw style deep red car, front view', # 프롬프트
            'dreamlike_lora_side2', # 결과 폴더 
            #lora='../lora/results/230402-05-bmw-dreamlike/checkpoint-000980/', # lora 사용
            scheduler = 'unipc',
            new_w = 512, # fix
            new_h = 768, # fix
            negative_prompt = '',
            guide_scale = 10.,
            cond_scale = 1.6,
            num_infer_steps = 30,
            num_images = 4,
            seed = -1,
            )

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [40]:
# with lora 768 x 512
module.run( 'front1.png', # 샘플 이미지
            'BMWSTYLE deep red car, front view', # 프롬프트
            '.', # 결과 폴더 
            lora='../lora/results/230402-05-bmw-dreamlike/checkpoint-000980/', # lora 사용
            scheduler = 'unipc',
            new_w = 512, # fix
            new_h = 768, # fix
            negative_prompt = '',
            guide_scale = 10.,
            cond_scale = 1.6,
            num_infer_steps = 30,
            num_images = 4,
            seed = -1,
            )

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]